# Chapter4. 구조적 API 개요

구조적 API의 세 가지 분산 컬렉션 API
* Dataset
* DataFrame
* SQL 테이블과 뷰

배치와 스트리밍 처리에서 구조적 API를 사용할 수 있다. 구조적 API를 활용하면 배치 작업을 스트리밍 작업으로 손쉽게 변환 가능. 

<학습할 내용>
* 타입형/비타입형 API의 개념과 차이점
* 핵심 용어
* 스파크가 구조적 API의 데이터 흐름을 해석하고 클러스터에서 실행하는 방식

## 4.1 DataFrame과 Dataset

스파크가 가지는 두 가지 구조화된 컬렉션 개념 : DataFrame & Dataset

* DataFrame & Dataset : 잘 정의된 로우와 컬럼을 가지는 분산 테이블 형태의 컬렉션
* 각 컬럼은 다른 컬럼과 동일한 수의 로우를 가져야 하고, 값이 없는 경우 null로 표시
* 컬렉션의 모든 로우는 같은 데이터 타입 정보를 가져야 함 
* DataFrame & Dataset은 결과를 생성하기 위해 어떤 데이터에 어떤 연산을 적용해야 하는지 정의하는 지연 연산의 실행 계획이며 불변서응ㄹ 가짐
* DataFrame에 액션을 호출하면 스파크는 트랜스포메이션을 실제로 실행하고 결과 반환

## 4.2 스키마

스키마 : DataFrame의 컬럼명과 데이터 타입 정의

* 스키마는 데이터소스에서 얻거나 직접 정의 가능
* 여러 데이터 타입으로 구성되므로 어떤 데이터 타입이 어느 위치에 있는지 정의하는 방법 필요

## 4.3 스파크의 구조적 데이터 타입 개요

스파크는 실행 계획 수립과 처리에 사용하는 자체 데이터 타입 정보를 가지고 있는 카탈리스트 엔진 사용.

카탈리스트 엔진은 다양한 실행 최적화 기능 제공. 

스파크는 자체 데이터 타입을 지원하는 여러 언어 API와 직접 매핑되고 각 언어에 대한 매핑 테이블을 가짐.

예제) 스파크의 덧셈 연산 수행

In [0]:
df = spark.range(500).toDF("number")
df.select(df["number"] + 10)

Out[1]: DataFrame[(number + 10): bigint]

### 4.3.1 DataFrame과 Dataset 비교

구조적 API에는 **비타입형 DataFrame** & **타입형 Dataset** 존재

< 스파크의 DataFrame >
* 스키마에 명시된 데이터 타입의 일치 여부를 런타임이 되어서야 확인 가능
* Row 타입으로 구성된 Dataset (Row 타입 : 스파크가 사용하는 연산에 최적화된 인메모리 포맷의 내부적인 표현 방식)
* 파이썬이나 R에서는 스파크의 Dataset을 사용할 수 없지만 최적화된 포맷인 DataFrame으로 처리 가능

< 스파크의 Dataset >
* 스키마에 명시된 데이터 타입의 일치 여부를 컴파일 타임에 확인
* JVM 기반의 언어인 스칼라와 자바에서만 지원
* 데이터 타입을 정의하려면 스칼라의 케이스 클래스(case class)나 자바 빈(JavaBean)을 사용

### 4.3.2 컬럼

컬럼의 표현
* 정수형이나 문자열 같은 단순 데이터 타입
* 배열이나 맵 같은 복합 데이터 타입
* null 값

스파크는 데이터 타입의 모든 정보를 추적하며 다양한 컬럼 변환 방법을 제공

### 4.3.3 로우

로우는 데이터의 레코드로, DataFrame의 레코드는 Row 타입으로 구성.

로우는 SQL, RDD, 데이터소스에서 얻거나 직접 생성 가능. 

예제) range 메서드를 사용해 DataFrame 생성 (Row 객체로 이루어진 배열 반환)

In [0]:
spark.range(2).collect()

Out[2]: [Row(id=0), Row(id=1)]

### 4.3.4 스파크 데이터 타입

In [0]:
from pyspark.sql.types import *

b = ByteType()

## 4.4 구조적 API의 실행 과정

구조적 API 쿼리가 사용자 코드에서 실제 실행 코드로 변환되는 과정
1. DataFrame/Dataset/SQL을 이용해 코드 작성
2. 정상적인 코드라면 스파크가 논리적 실행 계획으로 변환
3. 스파크는 논리적 실행 계획을 물리적 실행 계획으로 변환하며 그 과정에서 추가적인 최적화를 할 수 있는지 확인
4. 스파크는 클러스터에서 물리적 실행 계획(RDD  처리)을 실행

### 4.4.1 논리적 실행 계획

논리적 실행 계획 단계에서는 추상적 트랜스포메이션만 표현. 
* 드라이버나 익스큐터의 정보를 고려하지 않음
* 사용자의 다양한 표현식을 최적화된 버전으로 변환
* 사용자 코드는 검증 전 논리적 실행 계획으로 변환 (코드의 유효성과 테이블이나 컬럼의 존재 여부만을 판단하는 과정이므로 아직 실행 계획을 검증하지 않은 상태)
* 스파크 분석기는 컬럼과 테이블을 검증하기 위해 카탈로그, 모든 테이블의 저장소 그리고 DataFrame 정보 활용
* 필요한 테이블이나 컬럼이 카탈로그에 없다면 검증 전 논리적 실행 계획이 만들어지지 않음
* 테이블과 컬럼에 대한 검증 결과는 카탈리스트 옵티마이저로 전달

### 4.4.2 물리적 실행 계회

물리적 실행 계획은 논리적 실행 계획을 클러스터 환경에서 실행하는 방법 정의.
* 다양한 물리적 실행 전량을 생성하고 비용 모델을 이용해 비교한 후 최적의 전략 선택
* 일련의 RDD와 트랜스포메이션으로 변환

### 4.4.3 실행

1. 스파크는 물리적 실행 계획을 선정한 다음 저수준 프로그래밍 인터페이스인 RDD룰 대상으로 모든 코드 실행.

2. 스파크는 런타임에 전체 태스크나 스테이지를 제거할 수 있는 자바 바이트 코드를 생성해 추가적인 최적화 수행.

3. 스파크는 처리 결과를 사용자에게 반환